# Rachunek macierzowy - mnożenie macierzy

**Wykonali: Alicja Niewiadomska, Paweł Kruczkiewicz**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time


Wybrano temat nr 3, czyli *dla macierzy mniejszych lub równych 2^l x 2^l mnożenie rekurencyjne metodą Bineta, dla większych - Strassena*.


Wybrane algorytmy macierzowe:
 1. Mnożenie rekurencyjne metodą Bineta.
 2. Mnożenie rekurencyjne metodą Strassena.
 

## Pseudokod rozwiązania

Zakładamy, że dana macierz ma wymiary 2^k x 2^k.

Dane: 
  - `A`, `B` - macierze kwadratowe o rozmiarze 2^k
  - `k` - wyżej wspomniany wykładnik
  - `l` - arbitralnie wybrana eksponenta wartości progowej dla danego algorytmu.
  
Wartość zwracana: 
   `C` - wynikowa macierz kwadratowa o rozmiarze 2^k
```
def mat_mul(A, B, k, l):
    if l <= k:
        return binet_rec_mat_mul(A, B, k, l)
    else:
        return strass_mat_mul(A, B, k, l)
```

## Kod algorytmu

### Funkcje pomocnicze

In [ ]:
def split_matrix_in_quarters(A: np.array):
    half_i = A.shape[0] // 2
    return  A[:half_i, :half_i], \
            A[:half_i, half_i:], \
            A[half_i:, :half_i],\
            A[half_i:, half_i:]
            
def join_matrices(A11, A12, A21, A22):
    A1 = np.hstack((A11, A12))
    A2 = np.hstack((A21, A22))
    return np.vstack((A1, A2))

### Algorytm rekurencyjny Bineta

In [ ]:
def binet_rec_mat_mul(A: np.array, B: np.array, k: int, l: int) -> np.array:
    if k == 0:
        return A*B
    
    rec_step = lambda A1, B1, A2, B2: mat_mul(A1, B1, k-1, l) + mat_mul(A2, B2, k-1, l)

    A11, A12, A21, A22 = split_matrix_in_quarters(A)
    B11, B12, B21, B22 = split_matrix_in_quarters(B)
    
    C11 = rec_step(A11, B11, A12, B21)
    C12 = rec_step(A11, B12, A12, B22)
    C21 = rec_step(A21, B11, A22, B21)
    C22 = rec_step(A21, B12, A22, B22)
    
    return join_matrices(C11, C12, C21, C22)

### Algorytm rekurencyjny Strassena

In [ ]:
def strassen_mat_mul(A: np.array, B: np.array, k: int, l: int) -> np.array:    
    if k == 0:
        return A*B
    
    rec_step = lambda A1, B1: mat_mul(A1, B1, k-1, l)

    A11, A12, A21, A22 = split_matrix_in_quarters(A)
    B11, B12, B21, B22 = split_matrix_in_quarters(B)

    M1 = rec_step(A11 + A22, B11 + B22)
    M2 = rec_step(A21 + A22, B11)
    M3 = rec_step(A11, B12 - B22)
    M4 = rec_step(A22, B21 - B11)
    M5 = rec_step(A11 + A12, B22)
    M6 = rec_step(A21 - A11, B11 + B12)
    M7 = rec_step(A12 - A22, B21 + B22)

    C11 = M1 + M4 - M5 + M7
    C12 = M3 + M5
    C21 = M2 + M4
    C22 = M1 - M2 + M3 + M6
    
    return join_matrices(C11, C12, C21, C22)

### Algorytm końcowy

In [ ]:
def mat_mul(A, B, k, l):
    if l <= k:
        return binet_rec_mat_mul(A, B, k, l)
    else:
        return strassen_mat_mul(A, B, k, l)

## Wykresy
Wybrane wielkości parametru `l`: 4, 6, 8

In [ ]:
plt.rcParams["figure.figsize"] = [15, 10]
plt.rcParams['font.size'] = 24
plt.style.use('fivethirtyeight')

### Wykres czasu mnożenia od wielkości macierzy

In [ ]:
k_values = np.arange(2, 7)
l_values = np.arange(2, 7, 2)
num_range = 100

### kod generujący czas

def measure_mat_mul_time() -> np.array:
    times = []

    for i, k in enumerate(k_values):
        times.append([])
        for j, l in enumerate(l_values):
            size = 2**k
            A = np.random.randint(0, num_range, (size, size))
            B = np.random.randint(0, num_range, (size, size))
            print("Measuring: k=", k, " l=", l)
            
            # Multiplication time is too small for these
            if k < 4:
                it = 100
                start = time.time()
                for _ in range(it):
                    mat_mul(A, B, k, l)
                end = time.time()
                res = (end-start)/it
            else:
                start = time.time()
                mat_mul(A, B, k, l)
                end = time.time()
                res = end-start

            times[i].append(res)
    return np.array(times)

### kod wykresu

def plot_time_results(times: np.array) -> None:
    x = k_values
    x_axis = np.arange(len(x))
    
    for i, l in enumerate(l_values):
        plt.bar(x_axis + 0.1*i, times[i], 0.1, label=l)

    plt.xticks(x_axis, x)
    plt.xlabel("k (matrix size 2^k)")
    plt.ylabel("Time in log scale [s]")
    plt.yscale("log")
    plt.title("Matrices multiplication performance")
    plt.legend(title="Parameter l")
    plt.show()

In [ ]:
times = measure_mat_mul_time()
plot_time_results(times.T)

### Wykres wykonanych obliczeń zmiennoprzecinkowych w zależności od wielkości macierzy

In [ ]:
### kod generujący liczbę obliczeń zmiennoprzecinkowych

def binet_rec_mat_mul_op_count(k: int, l: int) -> np.array:
    if k == 0:
        return 1  # multiplying matrices
     
    rec_step = lambda: mat_mul_op_count(k-1, l) + mat_mul_op_count(k-1, l) + 1 # recursive step plus one addition
    
    C11_op_count = rec_step()
    C12_op_count = rec_step()
    C21_op_count = rec_step()
    C22_op_count = rec_step()
    
    return C11_op_count + C12_op_count + C21_op_count + C22_op_count
    

### kod wykresu